# Практическое задание 2



## Замечания

* Задание необходимо сдать боту до 06.12.2021
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (первое практическое задание)
2. Реализация градиентов по входу и распространения градиента по сети (back propagation)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод 
- обратный вывод, производные по входу и по параметрам

In [1]:
class Layer(object):
    def __init__(self):
        self.name = "Layer"

    def forward(self, input_data):
        pass

    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]

    def grad_x(self, input_data):
        pass

    def grad_param(self, input_data):
        return []

    def update_param(self, grads, learning_rate):
        pass

1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm


class Network(object):
    def __init__(self, layers, loss=None):
        self.name = "Network"
        self.layers = layers
        self.loss = loss

    def forward(self, input_data):
        return self.predict(input_data)

    def grad_x(self, input_data, labels):
        temp_input = input_data
        grad_layers = []

        for layer in self.layers:
            grad_layers.append(layer.grad_x(temp_input))
            temp_input = layer.forward(temp_input)

        loss_grad = self.loss.grad_x(temp_input, labels)  # по софтмаксу и лейблам
        total_grad = loss_grad.copy()
        total_grad = np.expand_dims(total_grad, 1)

        for grad in grad_layers[::-1]:
            total_grad = total_grad @ grad

        return np.squeeze(total_grad)

    def grad_param(self, input_data, labels):
        weights_grad = []
        temp_w_grads = []
        temp_input = input_data
        grad_layers = []
        for layer in self.layers:
            grad_layers.append(layer.grad_x(temp_input))
            if layer.name == "Dense":
                temp_w_grads.append(
                    (layer.grad_W(temp_input), layer.grad_b(temp_input))
                )
            else:
                temp_w_grads.append(None)
            temp_input = layer.forward(temp_input)
        loss_grad = self.loss.grad_x(temp_input, labels)
        total_grad = loss_grad.copy()
        total_grad = np.expand_dims(total_grad, 1)

        for i, layer in enumerate(self.layers[::-1]):
            if layer.name == "Dense":
                w_grad = total_grad @ temp_w_grads[len(self.layers) - 1 - i][0]
                b_grad = total_grad @ temp_w_grads[len(self.layers) - 1 - i][1]
                b_grad = np.squeeze(b_grad)
                w_grad = np.reshape(w_grad, (w_grad.shape[0], -1, b_grad.shape[1]))
                weights_grad.append((w_grad.mean(axis=0), b_grad.mean(axis=0)))
            else:
                weights_grad.append(None)
            total_grad = total_grad @ grad_layers[len(grad_layers) - 1 - i]

        return weights_grad[::-1]

    def update(self, grad_list, learning_rate):
        for i, layer in enumerate(self.layers):
            if layer.name == "Dense":
                layer.W = layer.W - learning_rate * grad_list[i][0]
                layer.b = layer.b - learning_rate * grad_list[i][1]

    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)
        return current_input

    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.predict(input_data), labels)

    def train_step(self, input_data, labels, learning_rate=0.001):
        grad_list = self.grad_param(input_data, labels)
        self.update(grad_list, learning_rate)

    def fit(
        self,
        trainX,
        trainY,
        validation_split=0.25,
        batch_size=1,
        nb_epoch=1,
        learning_rate=0.01,
    ):

        train_x, val_x, train_y, val_y = train_test_split(
            trainX, trainY, test_size=validation_split, random_state=42
        )
        for epoch in range(nb_epoch):
            # train one epoch
            for i in tqdm(range(int(len(train_x) / batch_size))):
                batch_x = train_x[i * batch_size : (i + 1) * batch_size]
                batch_y = train_y[i * batch_size : (i + 1) * batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            # validate
            val_accuracy = self.evaluate(val_x, val_y)
            print("%d epoch: val %.2f" % (epoch + 1, val_accuracy))

    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)
        y_true = np.argmax(testY, axis=1)
        val_accuracy = np.sum((y_pred == y_true)) / (len(y_true))
        return val_accuracy

#### 1.1 (6 баллов) Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer
- MaxPooling

In [3]:
# импорты
import numpy as np

In [4]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = "Dense"
        self.input_dim = input_dim
        self.output_dim = output_dim
        if W_init is None or b_init is None:
            self.W = np.random.uniform(
                low=(-np.sqrt(6) / np.sqrt(input_dim + output_dim)),
                high=(np.sqrt(6) / np.sqrt(input_dim + output_dim)),
                size=(input_dim, output_dim),
            )
            self.b = np.zeros(output_dim, "float32")
        else:
            self.W = W_init
            self.b = b_init

    def forward(self, input_data):
        assert input_data.shape[1] == self.W.shape[0], "Mismatch in dimensions"
        out = input_data @ self.W + self.b
        return out

    def grad_x(self, input_data):
        batch_size = input_data.shape[0]
        ans = np.empty((input_data.shape[0], self.W.shape[1], self.W.shape[0]))
        for i in range(ans.shape[0]):
            ans[i] = np.copy(self.W.T)
        return ans

        return self.W.T

    def grad_b(self, input_data):
        ans = np.empty((input_data.shape[0], len(self.b), len(self.b)))
        for i in range(len(ans)):
            ans[i] = np.eye(len(self.b))
        return ans

    def grad_W(self, input_data):
        ans = np.zeros(
            (len(input_data), self.W.shape[1], self.W.shape[0] * self.W.shape[1])
        )
        step = self.W.shape[1]
        for b in range(input_data.shape[0]):
            for i in range(self.W.shape[1]):
                for j in range(input_data.shape[1]):
                    ans[b][i][i + j * step] = input_data[b][j]
        return ans

    def update_W(self, grad, learning_rate):
        self.W -= learning_rate * np.mean(grad, axis=0).reshape(self.W.shape)

    def update_b(self, grad, learning_rate):
        self.b -= learning_rate * np.mean(grad, axis=0)

    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)

    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]


class ReLU(Layer):
    def __init__(self):
        self.name = "ReLU"

    def forward(self, input_data):
        input_data[input_data < 0] = 0.0
        return input_data

    def grad_x(self, input_data):
        ans = np.zeros((input_data.shape[0], input_data.shape[1], input_data.shape[1]))
        for b in range(ans.shape[0]):
            for i in range(ans.shape[1]):
                if input_data[b][i] > 0:
                    ans[b][i][i] = 1.0
        return ans


class Softmax(Layer):
    def __init__(self):
        self.name = "Softmax"

    def forward(self, input_data):
        for i in range(len(input_data)):
            denominator = sum(np.exp(input_data[i]))
            input_data[i] = np.exp(input_data[i]) / denominator
        return input_data

    def grad_x(self, input_data):
        ans = np.empty((input_data.shape[0], input_data.shape[1], input_data.shape[1]))
        for b in range(ans.shape[0]):
            denominator = sum(np.exp(input_data[b]))
            for i in range(ans.shape[1]):
                for j in range(ans.shape[2]):
                    if i == j:
                        ans[b][i][j] = (
                            np.exp(input_data[b][i])
                            / denominator
                            * (denominator - np.exp(input_data[b][j]))
                            / denominator
                        )
                    else:
                        ans[b][i][j] = (
                            -np.exp(input_data[b][i])
                            / denominator
                            * np.exp(input_data[b][j])
                            / denominator
                        )

        return np.array(ans)


class FlattenLayer(Layer):
    def __init__(self):
        self.name = "Flatten"

    def forward(self, input_data):
        data = np.swapaxes(input_data, 1, 3)
        out = np.reshape(np.swapaxes(data, 1, 2), (data.shape[0], -1))
        return out

    def grad_x(self):
        pass


class MaxPooling(Layer):
    def __init__(self, pool_size=(2, 2), strides=2):
        self.name = "MaxPooling"
        self.pool_size = pool_size
        self.strides = 2

    def forward(self, input_data):
        out_h = int(
            (input_data.shape[2] - (self.pool_size[0] - 1) - 1) / self.strides + 1
        )
        out_w = int(
            (input_data.shape[3] - (self.pool_size[1] - 1) - 1) / self.strides + 1
        )
        out = np.empty((input_data.shape[0], input_data.shape[1], out_h, out_w))
        for i in range(out_h):
            for j in range(out_w):
                out[:, :, i, j] = np.max(
                    input_data[
                        :,
                        :,
                        i * self.strides : i * self.strides + self.pool_size[0],
                        j * self.strides : j * self.strides + self.pool_size[1],
                    ],
                    axis=(2, 3),
                )
        return out

    def grad_x(self):
        pass

#### 1.2 (3 балла) Реализуйте теперь свёртночный слой   (опционально)

In [5]:
class Conv2D(Layer):
    def __init__(
        self,
        kernel_size,
        input_channels,
        output_channels,
        padding="valid",
        stride=1,
        kernels_init=None,
        bias_init=None,
    ):
        self.name = "Conv2D"
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.padding = padding
        self.stride = stride
        if kernels_init is None or bias_init is None:
            pass
        else:
            self.kernel = kernels_init
            self.bias = bias_init

    def forward(self, input_data):
        if self.padding == "same":
            if input_data.shape[2] % self.stride == 0:
                pad_h = max(self.kernel_size - self.stride, 0)
            else:
                pad_h = max(self.kernel_size - (input_data.shape[2] % self.stride), 0)
            if input_data.shape[3] % self.stride == 0:
                pad_w = max(self.kernel_size - self.stride, 0)
            else:
                pad_w = max(self.kernel_size - (input_data.shape[3] % self.stride), 0)
            pad_top = pad_h // 2
            pad_bottom = pad_h - pad_top
            pad_left = pad_w // 2
            pad_right = pad_w - pad_left
            temp_input_data = np.pad(
                input_data,
                ((0, 0), (0, 0), (pad_top, pad_bottom), (pad_left, pad_right)),
            )
            temp_input_data[
                :,
                :,
                pad_top : pad_top + input_data.shape[2],
                pad_left : pad_left + input_data.shape[3],
            ] = input_data
        else:
            temp_input_data = input_data

        out_h = (
            temp_input_data.shape[2] - self.kernel_size + self.stride
        ) // self.stride
        out_w = (
            temp_input_data.shape[3] - self.kernel_size + self.stride
        ) // self.stride
        out = np.zeros((input_data.shape[0], self.output_channels, out_h, out_w))
        for oc in range(self.output_channels):
            for i in range(out_h):
                for j in range(out_w):
                    out[:, oc, i, j] = np.tensordot(
                        temp_input_data[
                            :,
                            :,
                            i * self.stride : i * self.stride + self.kernel_size,
                            j * self.stride : j * self.stride + self.kernel_size,
                        ],
                        self.kernel[:, :, :, oc],
                        axes=([2, 3, 1], [0, 1, 2]),
                    )

            out[:, oc] += self.bias[oc]
        return out

    def grad_x(self):
        pass

    def grad_kernel(self):
        pass

#### 1.4 Теперь настало время теста. 
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

#### Чтение данных

In [6]:
import numpy as np

np.random.seed(123)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255


Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


#### Подготовка моделей

In [7]:
from keras import layers
from keras.models import Model, Sequential


def get_keras_model():
    input_image = layers.Input(shape=(1, 28, 28))
    pool1 = layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(
        input_image
    )
    flatten = layers.Flatten()(pool1)
    dense1 = layers.Dense(10, activation="softmax")(flatten)
    model = Model(inputs=input_image, outputs=dense1)

    from tensorflow.keras.optimizers import SGD, Adam

    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

    history = model.fit(
        X_train, Y_train, validation_split=0.25, batch_size=32, epochs=2, verbose=1
    )
    return model

In [8]:
def get_our_model(keras_model):
    maxpool = MaxPooling(pool_size=(2, 2), strides=2)
    flatten = FlattenLayer()
    dense = DenseLayer(
        196,
        10,
        W_init=keras_model.get_weights()[0],
        b_init=keras_model.get_weights()[1],
    )
    softmax = Softmax()
    net = Network([maxpool, flatten, dense, softmax])
    return net

In [9]:
keras_model = get_keras_model()

Epoch 1/2
1407/1407 [==============================] - 9s 4ms/step - loss: 0.5695 - accuracy: 0.8494 - val_loss: 0.3838 - val_accuracy: 0.8889
Epoch 2/2
1407/1407 [==============================] - 6s 4ms/step - loss: 0.3763 - accuracy: 0.8915 - val_loss: 0.3481 - val_accuracy: 0.8983


In [10]:
our_model = get_our_model(keras_model)

In [11]:
print(keras_model.get_weights()[0].shape)
print(keras_model.get_weights()[1].shape)

(196, 10)
(10,)


In [12]:
keras_prediction = keras_model.predict(X_test)
our_model_prediction = our_model.predict(X_test)

In [13]:
keras_prediction[3]

array([9.9616027e-01, 2.1196460e-08, 4.4861872e-04, 2.1950193e-04,
       8.4384692e-06, 1.8447244e-03, 5.3466519e-04, 4.9427280e-04,
       1.4624924e-04, 1.4328833e-04], dtype=float32)

In [14]:
our_model_prediction[3]

array([9.96160221e-01, 2.11964139e-08, 4.48618175e-04, 2.19501887e-04,
       8.43846736e-06, 1.84472340e-03, 5.34665648e-04, 4.94273076e-04,
       1.46249244e-04, 1.43288358e-04])

In [15]:
if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print("Test PASSED")
else:
    print("Something went wrong!")

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

#### 2.1 (1 балл) Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [16]:
class CrossEntropy(object):
    def __init__(self, eps=0.00001):
        self.name = "CrossEntropy"
        self.eps = eps

    def forward(self, input_data, labels):
        assert input_data.shape == labels.shape, "shapes must be equal"
        ans = []
        for i in range(input_data.shape[0]):
            temp_ans = 0
            for j in range(input_data.shape[1]):
                temp_ans += -np.log(input_data[i][j] + self.eps) * labels[i][j]
            ans.append(temp_ans)
        return np.array(ans)

    def calculate_loss(self, input_data, labels):
        return self.forward(input_data, labels)

    def grad_x(self, input_data, labels):
        assert input_data.shape == labels.shape, "shapes must be equal"
        ans = []
        for i in range(input_data.shape[0]):
            grad = []
            for j in range(input_data.shape[1]):
                grad.append(-labels[i][j] / input_data[i][j])
            ans.append(grad)
        return np.array(ans)

#### 2.2 (2 баллa) Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [17]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (
            net.calculate_loss(x + delta, labels)
            - net.calculate_loss(x - delta, labels)
        ) / (2 * eps)
        right_answer.append(diff)
    return np.array(right_answer).T


def test_net(net):
    x = np.array([[1, 2, 3], [2, 3, 4]])
    labels = np.array([[0.3, 0.2, 0.5], [0.1, 0.7, 0.2]])
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    print("num_grad shape", num_grad.shape)
    print("grad shape", grad.shape)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)

In [18]:
loss = CrossEntropy()
test_net(loss)

num_grad shape (2, 3)
grad shape (2, 3)
Test PASSED


#### 2.3 (2 балла)   Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [19]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x - delta)) / (2 * eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T


def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    print("num_grad", num_grad.shape)
    print("grad", grad.shape)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)


def random_test_layer(layer):
    x = np.random.randn(3, 10)
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Random Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)

In [20]:
layer = Softmax()
test_layer(layer)
random_test_layer(layer)

num_grad (2, 3, 3)
grad (2, 3, 3)
Test PASSED
Random Test PASSED


#### 2.4 (5 баллов) Реализуйте метод grad_x для классов ReLU и DenseLayer

In [21]:
layer = ReLU()
test_layer(layer)
random_test_layer(layer)

num_grad (2, 3, 3)
grad (2, 3, 3)
Test PASSED
Random Test PASSED


In [22]:
layer = DenseLayer(3, 10)
test_layer(layer)

num_grad (2, 10, 3)
grad (2, 10, 3)
Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [23]:
net = Network(
    [DenseLayer(3, 10), ReLU(), DenseLayer(10, 3), Softmax()], loss=CrossEntropy()
)
test_net(net)

num_grad shape (2, 3)
grad shape (2, 3)
Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1 (4 балла) Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является отномерным вектором.

In [24]:
def numerical_grad_b(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b + delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b - delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2 * eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T


def test_grad_b():
    input_size = 3
    output_size = 4
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)


def random_test_grad_b():
    print("\nRandom test begins")
    input_size = np.random.randint(1, 100)
    output_size = np.random.randint(1, 100)
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((np.random.randint(1, 100), input_size))
    print("input_size", input_size)
    print("output_size", output_size)
    print("x shape", x.shape)

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Random Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)


test_grad_b()
random_test_grad_b()

Test PASSED

Random test begins
input_size 62
output_size 15
x shape (37, 62)
Random Test PASSED


In [25]:
def numerical_grad_W(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W + delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W - delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2 * eps)
            right_answer.append(diff.T)
    return np.array(right_answer).T


def test_grad_W():
    input_size = 3
    output_size = 4
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)

    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)


def random_test_W():
    print("\nRandom test begins")
    input_size = np.random.randint(1, 100)
    output_size = np.random.randint(1, 100)
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((np.random.randint(1, 100), input_size))

    print("input_size", input_size)
    print("output_size", output_size)
    print("x shape", x.shape)

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)

    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print("Random Test PASSED")
    else:
        print("Something went wrong!")
        print("Numerical grad is")
        print(num_grad)
        print("Your gradiend is ")
        print(grad)


test_grad_W()
random_test_W()

Test PASSED

Random test begins
input_size 12
output_size 14
x shape (96, 12)
Random Test PASSED


In [26]:
net = Network(
    [DenseLayer(3, 20), ReLU(), DenseLayer(20, 3), Softmax()], loss=CrossEntropy()
)
test_net(net)


x = np.array([[1, 2, 3], [2, 3, 4]])
labels = np.array([[0.3, 0.2, 0.5], [0.1, 0.7, 0.2]])
gr = net.grad_param(x, labels)

num_grad shape (2, 3)
grad shape (2, 3)
Test PASSED


In [27]:
for elem in gr:
    if elem is not None:
        print("weights gradient shape =", elem[0].shape)
        print("bias gradient shape =", elem[1].shape)

weights gradient shape = (3, 20)
bias gradient shape = (20,)
weights gradient shape = (20, 3)
bias gradient shape = (3,)


#### 3.2 (4 балла) Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя. 

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [ ]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(
    trainX[::3],
    Y_train[::3],
    validation_split=0.25,
    batch_size=16,
    nb_epoch=5,
    learning_rate=0.01,
)

100%|██████████| 937/937 [03:08<00:00,  4.97it/s]


1 epoch: val 0.70


100%|██████████| 937/937 [03:08<00:00,  4.97it/s]


2 epoch: val 0.79


100%|██████████| 937/937 [03:07<00:00,  4.99it/s]


3 epoch: val 0.82


100%|██████████| 937/937 [03:08<00:00,  4.98it/s]


4 epoch: val 0.83


100%|██████████| 937/937 [03:08<00:00,  4.98it/s]

5 epoch: val 0.85


In [ ]:
net = Network(
    [DenseLayer(784, 20), ReLU(), DenseLayer(20, 10), Softmax()], loss=CrossEntropy()
)
trainX = X_train.reshape(len(X_train), -1)
net.fit(
    trainX[::6],
    Y_train[::6],
    validation_split=0.25,
    batch_size=16,
    nb_epoch=5,
    learning_rate=0.01,
)

  0%|▏                                                                                 | 1/468 [00:00<01:12,  6.42it/s]

1 epoch: val 0.79


  0%|▏                                                                                 | 1/468 [00:00<01:12,  6.46it/s]

2 epoch: val 0.84


  0%|▏                                                                                 | 1/468 [00:00<01:11,  6.54it/s]

3 epoch: val 0.86


  0%|▏                                                                                 | 1/468 [00:00<01:13,  6.33it/s]

4 epoch: val 0.87


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [01:14<00:00,  6.26it/s]

5 epoch: val 0.88


#### 3.5 (2 балла) Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети 

In [ ]:
net = Network(
    [
        DenseLayer(784, 128),
        ReLU(),
        DenseLayer(128, 64),
        ReLU(),
        DenseLayer(64, 20),
        ReLU(),
        DenseLayer(20, 10),
        Softmax(),
    ],
    loss=CrossEntropy(),
)
trainX = X_train.reshape(len(X_train), -1)
net.fit(
    trainX[::6],
    Y_train[::6],
    validation_split=0.25,
    batch_size=16,
    nb_epoch=5,
    learning_rate=0.01,
)

  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

1 epoch: val 0.74


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

2 epoch: val 0.84


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

3 epoch: val 0.88


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

4 epoch: val 0.89


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [15:53<00:00,  2.04s/it]

5 epoch: val 0.90
